In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import mahotas as mt
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
ds_path = "/home/khanna/Desktop/54_ML_in_agricultural_domain/Datasets/Leaf_Test"
img_files = os.listdir(ds_path)

In [ ]:
def create_dataset():
    names = ['SNo','area','perimeter','physiological_length','physiological_width','aspect_ratio','rectangularity','circularity', \
             'mean_r','mean_g','mean_b','stddev_r','stddev_g','stddev_b', \
             'contrast','correlation','inverse_difference_moments','entropy'
            ]
    df = pd.DataFrame([], columns=names)
#     print("hi")
    for i in range(1,321):
        imgpath = ds_path + "/" +str(i)+".jpg"
#         print(imgpath)
        main_img = cv2.imread(imgpath)
        if(main_img is None):
            #print("hi")
            b=0
            vector = [b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b];
        elif(main_img is not None):
#             print("not none")
            img = cv2.cvtColor(main_img, cv2.COLOR_BGR2RGB)         
            gs = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
#             print("1")
            blur = cv2.GaussianBlur(gs, (25,25),0)
            ret_otsu,im_bw_otsu = cv2.threshold(blur,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
#             print("2")
            kernel = np.ones((50,50),np.uint8)
            closing = cv2.morphologyEx(im_bw_otsu, cv2.MORPH_CLOSE, kernel)
            name=i
            #Shape features
            contours,hierarchy = cv2.findContours(closing,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
            cnt = contours[0]
            M = cv2.moments(cnt)
            area = cv2.contourArea(cnt)
            perimeter = cv2.arcLength(cnt,True)
            x,y,w,h = cv2.boundingRect(cnt)
            aspect_ratio = float(w)/h
#             print("3")
            if(area):
                #print("hi")
                rectangularity = w*h/area
                circularity = ((perimeter)**2)/area
            if(area==0):
                rectangularity = w*h
                circularity = ((perimeter)**2)
            #Color features
            red_channel = img[:,:,0]
            green_channel = img[:,:,1]
            blue_channel = img[:,:,2]
            blue_channel[blue_channel == 255] = 0
            green_channel[green_channel == 255] = 0
            red_channel[red_channel == 255] = 0
#             print("4")
            red_mean = np.mean(red_channel)
            green_mean = np.mean(green_channel)
            blue_mean = np.mean(blue_channel)

            red_std = np.std(red_channel)
            green_std = np.std(green_channel)
            blue_std = np.std(blue_channel)
#             print("5")
            #Texture features
            SNo = i
            #used to find textures in the pictures
            textures = mt.features.haralick(gs)
            ht_mean = textures.mean(axis=0)
            contrast = ht_mean[1]
            correlation = ht_mean[2]
            inverse_diff_moments = ht_mean[4]
            entropy = ht_mean[8]
#             print("6")
            vector = [SNo,area,perimeter,w,h,aspect_ratio,rectangularity,circularity,\
                      red_mean,green_mean,blue_mean,red_std,green_std,blue_std,\
                      contrast,correlation,inverse_diff_moments,entropy
                     ]
#             print("7")
            df_temp = pd.DataFrame([vector],columns=names)
#             print("8")
            df = df.append(df_temp)
            print(i)
    return df

In [ ]:
dataset = create_dataset()

In [ ]:
dataset.to_csv("trained_features.csv")

In [ ]:
def vector_calc(f):
    imgpath = ds_path + "/"+str(f)+".JPG"
    print(imgpath)
    main_img = cv2.imread(imgpath)
    #imread ki wajah se bgr format me image aati usse fir rgb or fir grayscale me convert krna h
    img = cv2.cvtColor(main_img, cv2.COLOR_BGR2RGB) 
    gs = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)

    #to reduce image noise and details 25 by 25 k  square me gaussian blur kiya h    
    blur = cv2.GaussianBlur(gs, (25,25),0)
    #https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_thresholding/py_thresholding.html
    ret_otsu,im_bw_otsu = cv2.threshold(blur,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

    kernel = np.ones((50,50),np.uint8)
    closing = cv2.morphologyEx(im_bw_otsu, cv2.MORPH_CLOSE, kernel)

    #Shape features
    contours,hierarchy = cv2.findContours(closing,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    cnt = contours[0]
    M = cv2.moments(cnt)
    area = cv2.contourArea(cnt)
    perimeter = cv2.arcLength(cnt,True)
    x,y,w,h = cv2.boundingRect(cnt)
    aspect_ratio = float(w)/h
    rectangularity = w*h/area
    circularity = ((perimeter)**2)/area

    #Color features
    red_channel = img[:,:,0]
    green_channel = img[:,:,1]
    blue_channel = img[:,:,2]
    blue_channel[blue_channel == 255] = 0
    green_channel[green_channel == 255] = 0
    red_channel[red_channel == 255] = 0

    red_mean = np.mean(red_channel)
    green_mean = np.mean(green_channel)
    blue_mean = np.mean(blue_channel)

    red_std = np.std(red_channel)
    green_std = np.std(green_channel)
    blue_std = np.std(blue_channel)

    #Texture features

    #used to find textures in the pictures
    textures = mt.features.haralick(gs)
    ht_mean = textures.mean(axis=0)
    contrast = ht_mean[1]
    correlation = ht_mean[2]
    inverse_diff_moments = ht_mean[4]
    entropy = ht_mean[8]

    vector = [area,perimeter,w,h,aspect_ratio,rectangularity,circularity,\
              red_mean,green_mean,blue_mean,red_std,green_std,blue_std,\
              contrast,correlation,inverse_diff_moments,entropy
             ]

    titles = ['Original Image','Grayscale',
                'Gaussian Blur', 'Otsu Thresholding','After Morphology']
    images = [main_img,gs,blur,im_bw_otsu,closing]

    for i in range(5):
        plt.subplot(2,3,i+1),plt.imshow(images[i],'gray')
        plt.title(titles[i])
        plt.xticks([]),plt.yticks([])
    plt.show() 

In [ ]:
f=input()
vector_calc(f)